In [2]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, filter_h, out_channels, max_length, filter_d=30, in_channels=1):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(filter_h, filter_d)), # check padding in originalcode
            #nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(max_length,1)))
        
    def forward(self, x):
        out = self.layer1(x)
        #out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        #out = self.fc(out)
        return out

class CUE_CNN(nn.Module):
    def __init__(self, filters, out_channels, max_length, hidden_units, drop_prob, user_size, num_classes=2):
        super(CUE_CNN, self).__init__()
        self.conv1 = ConvNet(filters[0], out_channels=out_channels, max_length=max_length - filters[0]  + 1)
        self.conv2 = ConvNet(filters[1], out_channels=out_channels, max_length=max_length - filters[1]  + 1)
        self.conv3 = ConvNet(filters[2], out_channels=out_channels, max_length=max_length - filters[2]  + 1)
        self.fc = nn.Sequential(
            nn.Linear(out_channels * 3 + user_size, hidden_units),
            nn.ReLU(), #dropout
            nn.Dropout(drop_prob),
            nn.Linear(hidden_units, num_classes))
    def forward(self, x, user_embedding):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out = torch.cat((out1, out2, out3, user_embedding), dim=1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

SyntaxError: invalid syntax (<ipython-input-2-6426222019e3>, line 46)

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from pdb import set_trace as brk

In [2]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, filter_h, out_channels, max_length, filter_d=30, in_channels=1):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(filter_h, filter_d)), # check padding in originalcode
            #nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(max_length,1)))
        
    def forward(self, x):
        out = self.layer1(x)
        #out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        #out = self.fc(out)
        return out

class CUE_CNN(nn.Module):
    def __init__(self, filters, out_channels, max_length, hidden_units, drop_prob, user_size, num_classes=2):
        super(CUE_CNN, self).__init__()
        self.conv1 = ConvNet(filters[0], out_channels=out_channels, max_length=max_length - filters[0]  + 1)
        self.conv2 = ConvNet(filters[1], out_channels=out_channels, max_length=max_length - filters[1]  + 1)
        self.conv3 = ConvNet(filters[2], out_channels=out_channels, max_length=max_length - filters[2]  + 1)
        self.fc = nn.Sequential(
            nn.Linear(out_channels * 3 + user_size, hidden_units),
            nn.ReLU(), #dropout
            nn.Dropout(drop_prob),
            nn.Linear(hidden_units, num_classes))
    def forward(self, x, user_embedding):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out = torch.cat((out1, out2, out3, user_embedding), dim=1)
        out = self.fc(out)
        return out

In [3]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CUE_CNN([1,3,5], 50, 10, 32, 0.3, 40).to(device)

input_1 = torch.rand((15, 1,  10, 30)).cuda()
input_2 = torch.rand((15, 40)).cuda()
labels = torch.randint(high=2,size=(15,1)).reshape((15,)).type(torch.LongTensor).cuda()

In [4]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

outputs = model(input_1, input_2)
loss = criterion(outputs, labels)

# Backward and optimize
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [5]:
outputs

tensor([[ 0.0470, -0.1802],
        [ 0.0069, -0.1949],
        [-0.0129, -0.1587],
        [-0.0538, -0.1911],
        [-0.0726, -0.2228],
        [ 0.0835, -0.2057],
        [-0.0621, -0.2741],
        [ 0.1404, -0.2220],
        [-0.0522, -0.1347],
        [ 0.1780, -0.2645],
        [ 0.0874, -0.2378],
        [ 0.0125, -0.2516],
        [ 0.0497, -0.1250],
        [-0.0606, -0.1879],
        [ 0.1768, -0.2426]], device='cuda:0')